In [1]:
from collections import OrderedDict
import time
from random import randint
import zmq
import itertools
import logging
import sys
from threading import Thread
from time import sleep

Queue Code

In [2]:
HEARTBEAT_LIVENESS = 3     # 3..5 is reasonable
HEARTBEAT_INTERVAL = 1.0   # Seconds

In [3]:
class Worker(object):
    def __init__(self, address):
        self.address = address
        self.expiry = time.time() + HEARTBEAT_INTERVAL * HEARTBEAT_LIVENESS

class WorkerQueue(object):
    def __init__(self):
        self.queue = OrderedDict()

    def ready(self, worker):
        self.queue.pop(worker.address, None)
        self.queue[worker.address] = worker

    def purge(self):
        """Look for & kill expired workers."""
        t = time.time()
        expired = []
        for address, worker in self.queue.items():
            if t > worker.expiry:  # Worker expired
                expired.append(address)
        for address in expired:
            print("Idle worker expired: %s" % address)
            self.queue.pop(address, None)

    def next(self):
        address, worker = self.queue.popitem(False)
        return address
    
    def is_empty(self):
        return len(self.queue) == 0

In [4]:

def run_message_queue():


    #  Paranoid Pirate Protocol constants
    PPP_READY = b"\x01"      # Signals worker is ready
    PPP_HEARTBEAT = b"\x02"  # Signals worker heartbeat
    identity = b"%04X-%04X" % (randint(0, 0x10000), randint(0, 0x10000))


    context = zmq.Context(1)

    backend = context.socket(zmq.ROUTER)  # ROUTER
    backend.bind("tcp://*:5556")  # For workers

    poll_workers = zmq.Poller()
    poll_workers.register(backend, zmq.POLLIN)
    workers = WorkerQueue()

    heartbeat_at = time.time() + HEARTBEAT_INTERVAL
    message_queue = []

    while True:

        socks = dict(poll_workers.poll(HEARTBEAT_INTERVAL * 1000))
        
        if randint(0, 4) == 0:
            msg = b'0'
            frames = [identity, b'', msg]
#             frames = []
            message_queue = [frames] + message_queue
            print(f" checking if can send message, {len(workers.queue)} active workers")

        
        if len(message_queue) != 0:
            if not workers.is_empty():
                print("SENDING MESSAGE")
                frames = message_queue.pop()
                frames.insert(0, workers.next())
                backend.send_multipart(frames)            

        # Handle worker message
        if socks.get(backend) == zmq.POLLIN:
            # Use worker address for LRU routing
            frames = backend.recv_multipart()
            if not frames:
                print("error")
                break

            # Validate control message, or return reply to client
            msg = frames[1:]
            if len(msg) == 1:
                address = frames[0]
#                 print("Setting worker to ready")
                workers.ready(Worker(address))
                if msg[0] not in (PPP_READY, PPP_HEARTBEAT):
                    print("E: Invalid message from worker: %s" % msg)
            else:
                print("RECEIVED RESPONSE", msg)
#                 frontend.send_multipart(msg)

            # Send heartbeats to idle workers if it's time
            if time.time() >= heartbeat_at:
                for worker in workers.queue:
                    msg = [worker, PPP_HEARTBEAT]
                    backend.send_multipart(msg)
                heartbeat_at = time.time() + HEARTBEAT_INTERVAL

        workers.purge()
    print("SERVER IS CLOSING")

Worker code

In [5]:
def run_consumer():
    HEARTBEAT_LIVENESS = 3
    HEARTBEAT_INTERVAL = 1
    INTERVAL_INIT = 1
    INTERVAL_MAX = 32

    #  Paranoid Pirate Protocol constants
    PPP_READY = b"\x01"      # Signals worker is ready
    PPP_HEARTBEAT = b"\x02"  # Signals worker heartbeat

    def worker_socket(context, poller):
        """Helper function that returns a new configured socket
           connected to the Paranoid Pirate queue"""
        worker = context.socket(zmq.DEALER) # DEALER
        identity = b"%04X-%04X" % (randint(0, 0x10000), randint(0, 0x10000))
        worker.setsockopt(zmq.IDENTITY, identity)
        poller.register(worker, zmq.POLLIN)
        worker.connect("tcp://localhost:5556")
        worker.send(PPP_READY)
        return worker

    context = zmq.Context(1)
    poller = zmq.Poller()

    liveness = HEARTBEAT_LIVENESS
    interval = INTERVAL_INIT

    heartbeat_at = time.time() + HEARTBEAT_INTERVAL

    worker = worker_socket(context, poller)
    while True:
        socks = dict(poller.poll(HEARTBEAT_INTERVAL * 1000))

        # Handle worker activity on backend
        if socks.get(worker) == zmq.POLLIN:
            #  Get message
            #  - 3-part envelope + content -> request
            #  - 1-part HEARTBEAT -> heartbeat
            frames = worker.recv_multipart()
            if not frames:
                break # Interrupted
            
            # get normal message
            if len(frames) == 3:
                print("Worker: Received a job", frames)
                worker.send_multipart(frames)
                liveness = HEARTBEAT_LIVENESS
                time.sleep(5)  # Do some heavy work
            # process heartbeat
            elif len(frames) == 1 and frames[0] == PPP_HEARTBEAT:
#                 print("Worker: received Queue heartbeat")
                liveness = HEARTBEAT_LIVENESS
            # process wrong message
            else:
                print("E: Invalid message: %s" % frames)
            interval = INTERVAL_INIT
        # process silence
        else:
            liveness -= 1
            if liveness == 0:
                print("W: Heartbeat failure, can't reach queue")
                print("W: Reconnecting in %0.2fs..." % interval)
                time.sleep(interval)

                if interval < INTERVAL_MAX:
                    interval *= 2
                poller.unregister(worker)
                worker.setsockopt(zmq.LINGER, 0)
                worker.close()
                worker = worker_socket(context, poller)
                liveness = HEARTBEAT_LIVENESS
        # send heartbeat
        if time.time() > heartbeat_at:
            heartbeat_at = time.time() + HEARTBEAT_INTERVAL
#             print("Worker: Sending Worker heartbeat")
            worker.send(PPP_HEARTBEAT)

In [6]:
# Thread(target=run_producer).start()
# Thread(target=run_producer).start()
Thread(target=run_message_queue).start()
sleep(1)
Thread(target=run_consumer).start()
Thread(target=run_consumer).start()
Thread(target=run_consumer).start()

sleep(60)

 checking if can send message, 0 active workers
SENDING MESSAGE
Worker: Received a job [b'5087-FD92', b'', b'0']
RECEIVED RESPONSE [b'5087-FD92', b'', b'0']
 checking if can send message, 2 active workers
SENDING MESSAGE
Worker: Received a job [b'5087-FD92', b'', b'0']
RECEIVED RESPONSE [b'5087-FD92', b'', b'0']
 checking if can send message, 2 active workers
SENDING MESSAGE
 checking if can send message, 2 active workers
SENDING MESSAGE
Worker: Received a job [b'5087-FD92', b'', b'0']
RECEIVED RESPONSE [b'5087-FD92', b'', b'0']
Worker: Received a job [b'5087-FD92', b'', b'0']
RECEIVED RESPONSE [b'5087-FD92', b'', b'0']
Idle worker expired: b'05EE-8347'
 checking if can send message, 2 active workers
SENDING MESSAGE
 checking if can send message, 2 active workers
SENDING MESSAGE
Worker: Received a job [b'5087-FD92', b'', b'0']
RECEIVED RESPONSE [b'5087-FD92', b'', b'0']
Worker: Received a job [b'5087-FD92', b'', b'0']
 checking if can send message, 3 active workers
SENDING MESSAGE
RECE